In [1]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib as tc
import gym
from random import sample
from tensorflow.keras.preprocessing.sequence import pad_sequences

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
gamma=0.99 
tau=0.001 
normalize_returns=False 
normalize_observations=True
batch_size=128 
observation_range=(0., 1000.) 
action_range=(-0.05, 0.05) 
return_range=(-np.inf, np.inf),
critic_l2_reg=0. 
actor_lr=3e-4 #1e-5
critic_lr=3e-5 #3e-5
clip_norm=None 
reward_scale=1.

env = gym.make("Pendulum-v0")

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
#state place holder
obs_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
action_high = env.action_space.high
action_low = env.action_space.low
print(action_high)
print(action_low)

#RNN network for state
with tf.compat.v1.variable_scope("state_encode"):
    obs_input = tf.keras.layers.Input(shape = (None,obs_dim))
    action_input = tf.keras.layers.Input(shape = (None,action_dim))
    #obs_h = tf.keras.layers.Dense(4,activation = 'relu')(obs_input)
    rnn_layer,state_h = tf.keras.layers.GRU(64,return_state=True)(tf.keras.layers.Concatenate()([action_input,obs_input]))
    print(obs_input)

#policy network
with tf.compat.v1.variable_scope("policy"):
    #output action
    action_output_h = tf.keras.layers.Dense(256,activation = 'relu')(state_h)
    action_output = tf.keras.layers.Dense(action_dim,activation = 'tanh')(action_output_h)*(action_high-action_low)/2+(action_low+action_high)/2
    
with tf.compat.v1.variable_scope("target_policy"):
    #output action
    target_action_output_h = tf.keras.layers.Dense(256,activation = 'relu')(state_h)
    target_action_output = tf.keras.layers.Dense(action_dim,activation = 'tanh')(action_output_h)*(action_high-action_low)/2+(action_low+action_high)/2

    
#output value function
with tf.compat.v1.variable_scope("value_function"):
    value_h = tf.keras.layers.Dense(4,activation = 'relu')(state_h)
    new_action_input = tf.keras.layers.Input(shape = (action_dim))
    value_input = tf.keras.layers.Concatenate()([new_action_input,value_h])
    value_h2 = tf.keras.layers.Dense(256,activation = 'relu')(value_input)
    Q_value_output = tf.keras.layers.Dense(1,activation = 'linear')(value_h2)
    
with tf.compat.v1.variable_scope("target_value_function"):
    target_value_h = tf.keras.layers.Dense(4,activation = 'relu')(state_h)
    target_new_action_input = tf.keras.layers.Input(shape = (action_dim))
    target_value_input = tf.keras.layers.Concatenate()([target_new_action_input, target_value_h])
    target_value_h2 = tf.keras.layers.Dense(256,activation = 'relu')(target_value_input)
    target_Q_value_output = tf.keras.layers.Dense(1,activation = 'linear')(target_value_h2)

#actor, critic model
model = tf.keras.Model([obs_input,action_input,new_action_input],[action_output,Q_value_output])
actor_model = tf.keras.Model([obs_input,action_input],action_output)
critic_model = tf.keras.Model([obs_input,action_input,new_action_input],Q_value_output)
target_actor_model = tf.keras.Model([obs_input,action_input],target_action_output)
target_critic_model = tf.keras.Model([obs_input,action_input,target_new_action_input],target_Q_value_output)
actor_model.summary()

[2.]
[-2.]
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Tensor("state_encode/input_1:0", shape=(?, ?, 3), dtype=float32)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 1)]    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, None, 4)      0           input_2[0][0]                    
                                                                 input_1[0][0]                  

In [4]:
#computation graph
#policy gradient ascent
Q_grad = tf.gradients(critic_model.output,new_action_input)
Q_action_grad = tf.compat.v1.placeholder(tf.float32, shape=(None,action_dim))
actor_model_weights = actor_model.trainable_weights
action_grad = tf.gradients(actor_model.output,actor_model_weights,-Q_action_grad)
act_grad_ascent = tf.train.AdamOptimizer(actor_lr).apply_gradients(zip(action_grad,actor_model_weights))

#critic mse
adam  = tf.keras.optimizers.Adam(critic_lr)
critic_model.compile(loss="mse", optimizer=adam)

#target networks
#moving average
ema = tf.train.ExponentialMovingAverage(decay=0.99)
policy_weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"policy")
value_weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"value_function")
target_policy_weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"target_policy")
target_value_weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"target_value_function")
target_policy_upd = ema.apply(policy_weights)
target_value_upd = ema.apply(value_weights)
target_policy_assign = tf.group([tf.assign(target_policy_weights[i], ema.average(policy_weights[i])) for i in range(len(policy_weights))])
target_value_assign = tf.group([tf.assign(target_value_weights[i], ema.average(value_weights[i])) for i in range(len(value_weights))])
#ema.average(critic_model_weights)


#initialization
sess = tf.Session()
sess.run(tf.global_variables_initializer())

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


In [5]:
EPOCH = 2000
max_step = 200
max_seq = 2
batch_size = 128
epsilon = 0.1

replay_buffer = []
max_size = 10000
rewards_history = []

for epoch in range(EPOCH):
    #print("epoch: ",epoch)
    obs_seq_list = []
    act_seq_list = []
    obs = env.reset()
    obs = obs.reshape((obs_dim,))
    obs_seq_list.append(obs)
    act_seq_list.append(np.zeros(action_dim))
    for t in range(max_step):
        #interaction
        obs_seq = np.asarray(obs_seq_list)
        act_seq = np.asarray(act_seq_list)
        obs_seq = obs_seq.reshape((1,-1,obs_dim))
        act_seq = act_seq.reshape((1,-1,action_dim))
        
        #pad input
        pad_width = max_seq-np.size(obs_seq,1)
        next_pad_width = max_seq-np.size(obs_seq,1)+1
        obs_seq = np.pad(obs_seq,((0,0),(pad_width,0),(0,0)))
        act_seq = np.pad(act_seq,((0,0),(pad_width,0),(0,0)))
        
        #choose action
        r = np.random.rand(1)
        if r>epsilon:
            action = sess.run(action_output,feed_dict={obs_input:obs_seq,action_input:act_seq})
        else:
            action = env.action_space.sample()
        
        action = action.reshape((action_dim,))
        next_obs,reward,done,_ = env.step(action)
        next_obs = next_obs.reshape((obs_dim,))
        
        obs_seq_list.append(next_obs)
        act_seq_list.append(action)
        #keep the length same
        if len(obs_seq_list)>max_seq:
            obs_seq_list.pop(0)
            act_seq_list.pop(0)
        next_obs_seq = np.asarray(obs_seq_list)
        next_act_seq = np.asarray(act_seq_list)
        next_obs_seq = next_obs_seq.reshape((1,-1,obs_dim))
        next_act_seq = next_act_seq.reshape((1,-1,action_dim))
        
        #pad data
        pad_width = max_seq-np.size(obs_seq,1)
        next_pad_width = max_seq-np.size(next_obs_seq,1)
        obs_seq = np.pad(obs_seq,((0,0),(pad_width,0),(0,0)))
        act_seq = np.pad(act_seq,((0,0),(pad_width,0),(0,0)))
        next_obs_seq = np.pad(next_obs_seq,((0,0),(next_pad_width,0),(0,0)))
        next_act_seq = np.pad(next_act_seq,((0,0),(next_pad_width,0),(0,0)))
        
        #record
        replay_buffer.append([obs_seq,act_seq,action,reward,next_obs_seq,next_act_seq,done])
        if len(replay_buffer)>max_size:
            replay_buffer.pop(0)
        
        #move to the next
        obs = next_obs
        
        
        #train
        #get data from the buffer
        if len(replay_buffer)>batch_size and t%5==0 and t!=0:
            train_datas = sample(replay_buffer,batch_size)
            array = np.asarray(train_datas)
            obs_seq_data = np.stack(array[:,0]).reshape((batch_size,max_seq,-1))
            act_seq_data = np.stack(array[:,1]).reshape((batch_size,max_seq,-1))
            action_data = np.stack(array[:,2]).reshape((batch_size,-1))
            reward_data = np.stack(array[:,3]).reshape((batch_size,-1))
            next_obs_data = np.stack(array[:,4]).reshape((batch_size,max_seq,-1))
            next_act_data = np.stack(array[:,5]).reshape((batch_size,max_seq,-1))
            done_data = np.stack(array[:,6]).reshape((batch_size,-1))

            #train actor
            new_action = sess.run(action_output, feed_dict={
                obs_input:  obs_seq_data,
                action_input: act_seq_data
            })
            Q_action_grads = sess.run(Q_grad, feed_dict={
                obs_input:  obs_seq_data,
                action_input: act_seq_data,
                new_action_input: new_action
            })[0]
            sess.run(act_grad_ascent, feed_dict={
                obs_input:  obs_seq_data,
                action_input: act_seq_data,
                Q_action_grad:Q_action_grads
            })
            #train critic
            target_actions = sess.run(target_action_output, feed_dict = {
                obs_input: next_obs_data,
                action_input: next_act_data,
            })
            target_Qs = sess.run(target_Q_value_output, feed_dict = {
                obs_input: next_obs_data,
                action_input: next_act_data,
                target_new_action_input: target_actions
            })
            Q_y = reward_data + gamma*target_Qs*(1-done)
            evaluation = critic_model.fit([obs_seq_data,act_seq_data,action_data], Q_y, verbose=0)
            if t==max_step-5:
                print(t,evaluation.history)
            #update target
            sess.run(target_policy_upd)
            sess.run(target_value_upd)
            sess.run(target_policy_assign)
            sess.run(target_value_assign)
        
        if done:
            break
                
    #evaluate
    if epoch%10==0:
        obs_seq_list = []
        act_seq_list = []
        ave_rewards = 0
        for times in range(5):
            rewards = 0
            obs = env.reset()          
            for t in range(500):
                #interaction
                obs_seq = np.asarray(obs_seq_list)
                act_seq = np.asarray(act_seq_list)
                obs_seq = obs_seq.reshape((1,-1,obs_dim))
                act_seq = act_seq.reshape((1,-1,action_dim))
                #pad input
                pad_width = max_seq-np.size(obs_seq,1)
                obs_seq = np.pad(obs_seq,((0,0),(pad_width,0),(0,0)))
                act_seq = np.pad(act_seq,((0,0),(pad_width,0),(0,0)))
                #choose action
                action = sess.run(action_output,feed_dict={obs_input:obs_seq,action_input:act_seq})
                action = action.reshape((action_dim,))
                next_obs,reward,done,_ = env.step(action)
                obs = next_obs.reshape((obs_dim,))
                obs_seq_list.append(obs)   
                act_seq_list.append(action)
                if len(obs_seq_list)>max_seq:
                    obs_seq_list.pop(0)
                    act_seq_list.pop(0)

                rewards = rewards+reward
                if done:
                    break
            ave_rewards = ave_rewards+rewards
            
        ave_rewards = ave_rewards/5
        rewards_history.append(ave_rewards)
        print("epoch: ",epoch)
        print("rewards: ", ave_rewards)

195 {'loss': [38.923171043395996]}
epoch:  0
rewards:  -1392.8579666364599
195 {'loss': [39.594122886657715]}
195 {'loss': [34.948400020599365]}
195 {'loss': [20.272123336791992]}
195 {'loss': [16.232497930526733]}
195 {'loss': [11.321239471435547]}
195 {'loss': [9.603323698043823]}
195 {'loss': [12.277170062065125]}
195 {'loss': [12.313477039337158]}
195 {'loss': [9.537736654281616]}
195 {'loss': [9.20673954486847]}
epoch:  10
rewards:  -1335.2337319279177
195 {'loss': [6.969679832458496]}
195 {'loss': [6.7913830280303955]}
195 {'loss': [4.385062158107758]}
195 {'loss': [4.255519151687622]}
195 {'loss': [3.6625298857688904]}
195 {'loss': [2.5010078251361847]}
195 {'loss': [1.3967384994029999]}
195 {'loss': [1.7011220753192902]}
195 {'loss': [1.4789334833621979]}
195 {'loss': [1.5104963183403015]}
epoch:  20
rewards:  -1406.34884227453
195 {'loss': [1.030065581202507]}
195 {'loss': [1.3937216252088547]}
195 {'loss': [1.4787666648626328]}
195 {'loss': [0.9364462792873383]}
195 {'loss': 

KeyboardInterrupt: 

In [ ]:
a = [0,1,2,3]
a.append(4)
print(a)
a.pop(0)
print(a)

##### 